<a href="https://colab.research.google.com/github/Caffeine-Coders/Sentiment-Analysis-Project/blob/main/backend/Roberta_custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/DatasetsforSentimentAnalysis/TweetsData.csv', header = None)
data[1].fillna("random", inplace=True)
data[3].fillna(0, inplace=True)
data[3] = data[3].replace("neutral",1)
data[3] = data[3].replace("negative",0)
data[3] = data[3].replace("positive",2)
data = data.drop(0)
data[3]= data[3].apply(np.float64)
data[3]= data[3].apply(np.int64)
# data = data.sample(frac = 1)
# data = data.sample(frac = 1)
# data = data.sample(frac = 1)
data = data.drop(columns = [0,2])
columns_names = list(data)
data.rename(columns={columns_names[0]:0,
                        columns_names[1]:1}, inplace= True)

data2 = data.loc[data[1] == 1]
data2 = pd.concat([data2, data2])
print(type(data2))
print(data2)
# tweets2 = data["text"].values.tolist()
# classifiers2 = data["sentiment"].values.tolist()

<class 'pandas.core.frame.DataFrame'>
                                                       0  1
1                    I`d have responded, if I were going  1
6      http://www.dothebouncy.com/smf - some shameles...  1
8                                             Soooo high  1
9                                            Both of you  1
11      as much as i love to be hopeful, i reckon the...  1
...                                                  ... ..
27469  few grilled mushrooms and olives, feta cheese ...  1
27470              94 more days till BH comes back to LA  1
27472  i`m defying gravity. and nobody in alll of oz,...  1
27474   in spoke to you yesterday and u didnt respond...  1
27481     All this flirting going on - The ATG smiles...  1

[22236 rows x 2 columns]


In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/DatasetsforSentimentAnalysis/sentiment140.csv', 
                        encoding='latin-1', header = None)
  
data.drop(data.columns[[1,2, 3,4]], axis=1, inplace=True)
data[5].fillna("random", inplace=True)
data[0].fillna(0, inplace=True)
data[0] = data[0].replace(2,1)
data[0] = data[0].replace(4,2)
data[0]= data[0].apply(np.int64)
columns_names = list(data)
print(columns_names)
data.rename(columns={columns_names[0]:0,
                        columns_names[1]:1}, inplace= True)
NUM_SAMPLES = 400000
negative_samples = data[data[0]==0][:NUM_SAMPLES]
positiv_samples = data[data[0]==2][:NUM_SAMPLES]

positiv_samples[0]=[1]*NUM_SAMPLES
full_data = pd.concat([negative_samples,  positiv_samples])
# data = full_data
# data = data.sample(frac = 1)
# data = data.sample(frac = 1)
# data = data.sample(frac = 1)
temp = data[0]
data[0] = data[1]
data[1] = temp
print(type(data))
data3 = data
print(type(data3))
print(data3)
# tweets3 = data[5].values.tolist() 
# classifiers3 = data[0].values.tolist()

[0, 5]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
                                                         0  1
0        @switchfoot http://twitpic.com/2y1zl - Awww, t...  0
1        is upset that he can't update his Facebook by ...  0
2        @Kenichan I dived many times for the ball. Man...  0
3          my whole body feels itchy and like its on fire   0
4        @nationwideclass no, it's not behaving at all....  0
...                                                    ... ..
1599995  Just woke up. Having no school is the best fee...  2
1599996  TheWDB.com - Very cool to hear old Walt interv...  2
1599997  Are you ready for your MoJo Makeover? Ask me f...  2
1599998  Happy 38th Birthday to my boo of alll time!!! ...  2
1599999  happy #charitytuesday @theNSPCC @SparksCharity...  2

[1600000 rows x 2 columns]


In [ ]:
print(data3.shape)
mergedata = pd.concat([data2, data3])
print(mergedata)

(1600000, 2)
                                                         0  1
1                      I`d have responded, if I were going  1
6        http://www.dothebouncy.com/smf - some shameles...  1
8                                               Soooo high  1
9                                              Both of you  1
11        as much as i love to be hopeful, i reckon the...  1
...                                                    ... ..
1599995  Just woke up. Having no school is the best fee...  2
1599996  TheWDB.com - Very cool to hear old Walt interv...  2
1599997  Are you ready for your MoJo Makeover? Ask me f...  2
1599998  Happy 38th Birthday to my boo of alll time!!! ...  2
1599999  happy #charitytuesday @theNSPCC @SparksCharity...  2

[1622236 rows x 2 columns]


In [ ]:
import pandas as pd
import io
import re

with open('/content/drive/MyDrive/DatasetsforSentimentAnalysis/book.txt', 'r') as file:
    data = file.read()

df1 = pd.read_csv(io.StringIO(data), delimiter="\t", header = None)

with open('/content/drive/MyDrive/DatasetsforSentimentAnalysis/kitchen.txt', 'r') as file:
    data = file.read()

df2 = pd.read_csv(io.StringIO(data), delimiter="\t", header = None)

with open('/content/drive/MyDrive/DatasetsforSentimentAnalysis/dvd.txt', 'r') as file:
    data = file.read()

df3 = pd.read_csv(io.StringIO(data), delimiter="\t", header = None)

with open('/content/drive/MyDrive/DatasetsforSentimentAnalysis/elec.txt', 'r') as file:
    data = file.read()

df4 = pd.read_csv(io.StringIO(data), delimiter="\t", header = None)

data1 = []
for line in df1[0]:
    obtained = line.split(" #label#:")
    obtained[0] = re.sub(r':\d+', ' ', obtained[0])
    obtained[0] = obtained[0].replace('_', ' ')
    data1.append(obtained)

data2 = []
for line in df2[0]:
    obtained = line.split(" #label#:")
    obtained[0] = re.sub(r':\d+', ' ', obtained[0])
    obtained[0] = obtained[0].replace('_', ' ')
    data2.append(obtained)

data3 = []
for line in df3[0]:
    obtained = line.split(" #label#:")
    obtained[0] = re.sub(r':\d+', ' ', obtained[0])
    obtained[0] = obtained[0].replace('_', ' ')
    data3.append(obtained)

data4 = []
for line in df4[0]:
    obtained = line.split(" #label#:")
    obtained[0] = re.sub(r':\d+', ' ', obtained[0])
    obtained[0] = obtained[0].replace('_', ' ')
    data4.append(obtained)

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)
df4 = pd.DataFrame(data4)

frame = [df1, df2, df3, df4]
enddata = pd.concat(frame)
enddata[0].fillna("random", inplace=True)
enddata[1].fillna('0', inplace=True)
enddata[1] = enddata[1].apply(np.float64)
enddata[1] = enddata[1].apply(np.int64)
enddata[1] = enddata[1].replace(1,0)
enddata[1] = enddata[1].replace(2,0)
enddata[1] = enddata[1].replace(4,2)
enddata[1] = enddata[1].replace(5,2)
enddata[1]= enddata[1].apply(np.int64)
# enddata = enddata.sample(frac = 1)
# enddata = enddata.sample(frac = 1)
# enddata = enddata.sample(frac = 1)
data4 = enddata
print(data4)
# tweets4 = enddata[0].values.tolist() 
# classifiers4 = enddata[1].values.tolist()

                                                      0  1
0     is ridiculous  me  who wants  or  of law  or l...  0
1     stand  to jackie  reason  whether  are  in  so...  0
2     contains numerous  omissio  numerous errors  a...  0
3     art  i think  think it  is written  on  by  it...  0
4     rainbow.you see  is simply  of simple  artistr...  0
...                                                 ... ..
1176  you load  whole purpose  look  bottom  intent ...  0
1177  i  selling model  the mini   purchase  inserts...  0
1178  of  always  time isnt  battery people  product...  0
1179  pleased  it the  are  within minutes  than wha...  2
1180  is  as the  a phone  nav-traffic  pioneer work...  0

[4336 rows x 2 columns]


In [ ]:
print(data4.shape)
mergedata = pd.concat([mergedata, data4])
print(mergedata)

(4336, 2)
                                                      0  1
1                   I`d have responded, if I were going  1
6     http://www.dothebouncy.com/smf - some shameles...  1
8                                            Soooo high  1
9                                           Both of you  1
11     as much as i love to be hopeful, i reckon the...  1
...                                                 ... ..
1176  you load  whole purpose  look  bottom  intent ...  0
1177  i  selling model  the mini   purchase  inserts...  0
1178  of  always  time isnt  battery people  product...  0
1179  pleased  it the  are  within minutes  than wha...  2
1180  is  as the  a phone  nav-traffic  pioneer work...  0

[1626572 rows x 2 columns]


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/DatasetsforSentimentAnalysis/Emojis.csv')
data["sentiment"] = data["sentiment"].replace("positive",2)
data["sentiment"] = data["sentiment"].replace("negative",0)
data["sentiment"] = data["sentiment"].replace("neutral",1)
print(data)
columns_names = list(data)
print(columns_names)
data.rename(columns={columns_names[0]:0,
                        columns_names[1]:1}, inplace= True)
print(data)
mergedata = pd.concat([mergedata, data])
print(mergedata)
# tweets5 = data["emoji"].values.tolist() 
# classifiers5 = data["sentiment"].values.tolist()



    emoji  sentiment
0       😂          1
1       ❤          2
2       ♥          2
3       😍          2
4       😭          1
..    ...        ...
964     ➛          1
965     ♝          1
966     ❋          1
967     ✆          2
968     📔          1

[969 rows x 2 columns]
['emoji', 'sentiment']
     0  1
0    😂  1
1    ❤  2
2    ♥  2
3    😍  2
4    😭  1
..  .. ..
964  ➛  1
965  ♝  1
966  ❋  1
967  ✆  2
968  📔  1

[969 rows x 2 columns]
                                                     0  1
1                  I`d have responded, if I were going  1
6    http://www.dothebouncy.com/smf - some shameles...  1
8                                           Soooo high  1
9                                          Both of you  1
11    as much as i love to be hopeful, i reckon the...  1
..                                                 ... ..
964                                                  ➛  1
965                                                  ♝  1
966                                

In [ ]:
print(mergedata.shape)
mergedata = mergedata.sample(frac = 1)
mergedata = mergedata.sample(frac = 1)
mergedata = mergedata.sample(frac = 1)
mergedata = mergedata.reset_index(drop = True)
print(mergedata)
tweets5 = mergedata[0].values.tolist() 
classifiers5 = mergedata[1].values.tolist()

(1627541, 2)
                                                         0  1
0        still can't get over that huge Superman collec...  0
1                              Off to mother's day brunch   2
2        wishing i can go to the CARLOS OLIVERO movie n...  0
3        gotta back to my (not so) beautiful physics bo...  0
4        @TamboManJoe do you know how important is for ...  0
...                                                    ... ..
1627536  @RandomGuyAndGal I had fun talking w/ you two ...  2
1627537       alone in my house..everybody's gone poor me   0
1627538  Just one more week...I think I can take it. Bu...  0
1627539       Half of the fish are dead or on the verge..   0
1627540                          @RetroPop I know! Me too   0

[1627541 rows x 2 columns]


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import datasets
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, TFTrainer, TFTrainingArguments
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base", num_labels=3)
print("------------------train and test splits --------------------------")
X1_train, X1_test, Y1_train, Y1_test = train_test_split(tweets5, classifiers5, test_size=0.18, random_state=42)
# X2_train, X2_test, Y2_train, Y2_test = train_test_split(tweets2, classifiers2, test_size=0.5, random_state=42)
# X3_train, X3_test, Y3_train, Y3_test = train_test_split(tweets3, classifiers3, test_size=0.49159, random_state=42)
# X4_train, X4_test, Y4_train, Y4test = train_test_split(tweets4, classifiers4, test_size=0.5, random_state=42)
# X5_train, X5_test, Y5_train, Y5_test = train_test_split(tweets5, classifiers5, test_size=0.05, random_state=42)
# print(len(X1_train))
# print(len(X2_train))
# print(len(X3_train))
# print(len(X4_train))
# # print(len(X5_train))
# X1_train.extend(X2_train)
# X1_train.extend(X3_train)
# X1_train.extend(X4_train)
# # X1_train.extend(X5_train)
# print(len(X1_train))
# Y1_train.extend(Y2_train)
# Y1_train.extend(Y3_train)
# Y1_train.extend(Y4_train)
# Y1_train.extend(Y5_train)
print(len(Y1_train), len(X1_train))
dataset = tf.data.Dataset.from_tensor_slices(X1_train)
dataset_test = tf.data.Dataset.from_tensor_slices(X1_test)

------------------train and test splits --------------------------
1334583 1334583


In [ ]:
print("-------------------tokenizing------------------------------")
train_encodings1 = tokenizer(X1_train, padding = "max_length", truncation = True ,return_tensors='np')

-------------------tokenizing------------------------------


In [ ]:
dataset = mergedata.map(train_encodings1)
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
tf_dataset = model.prepare_tf_dataset(dataset, batch_size = 128, shuffle = True, tokenizer = tokenizer)
train_labels1 = np.array(Y1_train).astype(np.int64)
test_labels1 = np.array(Y1_test).astype(np.int64)
opt = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)  # Transformers like lower learning rates
model.compile(optimizer=opt, metrics = [tf.keras.metrics.SparseCategoricalAccuracy(), tf.keras.metrics.Accuracy(), tf.keras.metrics.TopKCategoricalAccuracy(k = 3)])
early_stop = EarlyStopping(monitor = 'loss', patience = 2)

----------------------------Training happens here -----------------------------

In [ ]:
model.fit(dict(train_encodings1), train_labels1, epochs=150, callbacks = [early_stop], validatation_data = (dataset_test, test_labels1))
model.save_pretrained('/content/customRobertaModel')

In [ ]:
print("model is trained")

In [ ]:
!zip -r /content/customRobertaModel.zip /content/customRobertaModel/

In [ ]:
from google.colab import files
files.download("/content/customRobertaModel.zip")